In [21]:
pip install -U vnstock

Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [23]:
pip install ta

Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install keras-self-attention

Note: you may need to restart the kernel to use updated packages.


In [26]:
pip install keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [27]:
import numpy as np
import pandas as pd
from ta.momentum import RSIIndicator
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, RNN, concatenate
from tensorflow.keras.optimizers import RMSprop
from vnstock import *
from ta.volatility import BollingerBands
from tensorflow.keras.layers import Layer
import tensorflow as tf
from keras.layers import Input, Conv1D, MaxPooling1D, Bidirectional, Flatten, BatchNormalization
from sklearn.model_selection import RandomizedSearchCV
# from scikeras.wrappers import KerasClassifier

from ta.momentum import StochasticOscillator
from ta.volume import OnBalanceVolumeIndicator
from ta.trend import CCIIndicator
from sklearn.preprocessing import StandardScaler
from ta.volume import ChaikinMoneyFlowIndicator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.regularizers import l1
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras import regularizers
from keras_self_attention import SeqSelfAttention
from tensorflow.keras.models import Model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
import keras_tuner as kt
from tensorflow.keras.optimizers import Adam
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.base import BaseEstimator, ClassifierMixin
from tensorflow.keras.saving import register_keras_serializable
import matplotlib.pyplot as plt
# from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from vnstock import *
import pandas as pd
import os
from collections import Counter
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import RandomForestClassifier # Import RandomForestClassifier
from sklearn.metrics import accuracy_score # Import accuracy
from sklearn.metrics import log_loss
from sklearn.linear_model import LinearRegression
from xgboost import XGBClassifier
import optuna
import joblib
from sklearn.metrics import (
    f1_score, accuracy_score, classification_report, roc_auc_score,
    roc_curve, auc
)
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import TimeSeriesSplit

In [28]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [29]:
def dataframe_to_x_y(df, feature, target):
  X_train, y_train = [], []
  X_val, y_val = [], []
  n = len(df)
  split_index = int(n * 0.8)
  X_train = df[feature][:split_index]
  y_train = df[target][:split_index]
  X_val = df[feature][split_index:]
  y_val = df[target][split_index:]
  return X_train, y_train, X_val, y_val

In [30]:

sentiment_columns = [
        'Reputation', 'Company Communication', 'Appointment', 'Financial', 'Regulatory',
        'Sales', 'M&A', 'Legal', 'Dividend Policy', 'Risks', 'Rumors', 'Strategy',
        'Options', 'IPO', 'Signal', 'Coverage', 'Fundamentals', 'Insider Activity',
        'Price Action', 'Buyside', 'Technical Analysis', 'Trade', 'Central Banks',
        'Currency', 'Conditions', 'Market', 'Volatility', 'Investor Sentiment', 'Retail Investor Behavior',
        'Speculation', 'Domestic Institutional Behavior', 'Foreign Institutional Behavior', 'Black Swan Event',
    ]

sentiment_p1d_columns = [
        'reputation_p1d', 'company_communication_p1d', 'appointment_p1d', 'financial_p1d', 'regulatory_p1d',
        'sales_p1d', 'm&a_p1d', 'legal_p1d', 'dividend_policy_p1d', 'risks_p1d', 'rumors_p1d', 'strategy_p1d',
        'options_p1d', 'ipo_p1d', 'signal_p1d', 'coverage_p1d', 'fundamentals_p1d', 'insider_activity_p1d',
        'price_action_p1d', 'buyside_p1d', 'technical_analysis_p1d', 'trade_p1d', 'central_banks_p1d',
        'currency_p1d', 'conditions_p1d', 'market_p1d', 'volatility_p1d', 'investor_sentiment_p1d', 'retail_investor_behavior_p1d',
        'speculation_p1d', 'domestic_institutional_behavior_p1d', 'foreign_institutional_behavior_p1d', 'black_swan_event_p1d',
    ]
meta_cols = ['num_sa_news', 'mean_sentiment_norm', 'max_sentiment_norm', 'std_sentiment_score', 'sum_positive_sa', 'sum_negative_sa', 'time_distance_from_sa_to_close_minutes']
meta_p1d_cols = ['num_sa_news_p1d', 'mean_sentiment_norm_p1d', 'max_sentiment_norm_p1d', 'std_sentiment_score_p1d', 'sum_positive_sa_p1d', 'sum_negative_sa_p1d', 'time_distance_from_sa_to_close_minutes_p1d']

sentiment_feature_selected =  sentiment_columns + sentiment_p1d_columns + meta_cols + meta_p1d_cols
ta_features = ['volume_ma','volume_to_volume_ma_ratio','ema_12','ema_26','sma_20','sma_50','roc_5','roc_1','roc_9','%K','%R','cci','obv','macd','signal_line','macd_histogram','rsi','rsi_base_ma','rsi_rsi_base_ma_ratio','bb_bbm','bb_bbh','bb_bbl','bb_bbp','bb_bbh_bb_bbl_ratio','hl_ratio', 'co_ratio', 'price_range', 'sma_ratio_20_50', 'ema_ratio_12_26', 'bb_width', 'bb_position', 'rsi_overbought', 'rsi_oversold', 'rsi_neutral', 'macd_bullish', 'momentum_5', 'momentum_10','log_return','volatility_5d','volatility_10d','volatility_20d','volatility_30d','mean_log_return_5d','mean_log_return_10d','mean_log_return_20d','mean_log_return_30d','sharpe_like_5d','sharpe_like_10d','sharpe_like_20d','sharpe_like_30d','up_streak','pos_log_return_ratio_20d','z_score_5d','z_score_10d','z_score_20d','z_score_30d','annual_return','daily_return','sharpe_ratio',
               'rsi_vn30','rsi_base_ma_vn30','rsi_rsi_base_ma_ratio_vn30','volume_ma_vn30','volume_to_volume_ma_ratio_vn30','bb_bbm_vn30','bb_bbh_vn30','bb_bbl_vn30','bb_bbp_vn30','bb_bbh_bb_bbl_ratio_vn30','roc_1_vn30', 'roc_5_vn30', 'roc_9_vn30','%K_vn30','%R_vn30','cci_vn30','obv_vn30','ema_12_vn30','ema_26_vn30','sma_20_vn30','sma_50_vn30', 'hl_ratio_vn30', 'co_ratio_vn30', 'price_range_vn30', 'sma_ratio_20_50_vn30', 'ema_ratio_12_26_vn30', 'bb_width_vn30', 'bb_position_vn30', 'rsi_overbought_vn30', 'rsi_oversold_vn30', 'rsi_neutral_vn30', 'momentum_5_vn30', 'momentum_10_vn30', 'log_return_vn30','volatility_5d_vn30','volatility_10d_vn30','volatility_20d_vn30','volatility_30d_vn30','mean_log_return_5d_vn30','mean_log_return_10d_vn30','mean_log_return_20d_vn30','mean_log_return_30d_vn30','sharpe_like_5d_vn30','sharpe_like_10d_vn30','sharpe_like_20d_vn30','sharpe_like_30d_vn30','up_streak_vn30','pos_log_return_ratio_20d_vn30','z_score_5d_vn30','z_score_10d_vn30','z_score_20d_vn30','z_score_30d_vn30','annual_return_vn30','daily_return_vn30','sharpe_ratio_vn30',
               'rsi_vni','rsi_base_ma_vni','rsi_rsi_base_ma_ratio_vni','volume_ma_vni','volume_to_volume_ma_ratio_vni','bb_bbm_vni','bb_bbh_vni','bb_bbl_vni','bb_bbp_vni','bb_bbh_bb_bbl_ratio_vni','roc_1_vni', 'roc_5_vni', 'roc_9_vni','%K_vni','%R_vni','cci_vni','obv_vni','ema_12_vni','ema_26_vni','sma_20_vni','sma_50_vni', 'hl_ratio_vni', 'co_ratio_vni', 'price_range_vni', 'sma_ratio_20_50_vni', 'ema_ratio_12_26_vni', 'bb_width_vni', 'bb_position_vni', 'rsi_overbought_vni', 'rsi_oversold_vni', 'rsi_neutral_vni', 'momentum_5_vni', 'momentum_10_vni','log_return_vni','volatility_5d_vni','volatility_10d_vni','volatility_20d_vni','volatility_30d_vni','mean_log_return_5d_vni','mean_log_return_10d_vni','mean_log_return_20d_vni','mean_log_return_30d_vni','sharpe_like_5d_vni','sharpe_like_10d_vni','sharpe_like_20d_vni','sharpe_like_30d_vni','up_streak_vni','pos_log_return_ratio_20d_vni','z_score_5d_vni','z_score_10d_vni','z_score_20d_vni','z_score_30d_vni','annual_return_vni','daily_return_vni','sharpe_ratio_vni']
fa_features = ['p/b_previous_quarter', 'p/b_change_rate','p/b_change_rate_flag','p/e_previous_quarter','p/e_change_rate','p/e_change_rate_flag','p/s_previous_quarter','p/s_change_rate','p/s_change_rate_flag','p/cash_flow_previous_quarter','p/cash_flow_change_rate','p/cash_flow_change_rate_flag','eps_previous_quarter','eps_change_rate', 'eps_change_rate_flag','bvps_previous_quarter','bvps_change_rate', 'bvps_change_rate_flag','roe_previous_quarter','roe_change_rate','roe_change_rate_flag','roa_previous_quarter','roa_change_rate','roa_change_rate_flag','coefficient_p/b','coefficient_p/e','coefficient_p/s','coefficient_p/cash_flow','coefficient_eps','coefficient_bvps','coefficient_roe','coefficient_roa','distance_to_nearest_quarter']
ta_fa_feature_selected = ta_features + fa_features
features = ta_fa_feature_selected + sentiment_feature_selected
target = 'target'
scaler = StandardScaler()

In [31]:
def load_data_for_analysis(tickers, folder_path):

    df_all = []

    for ticker in tickers:
        file_path = os.path.join(folder_path, f"{ticker}.xlsx")
        if os.path.exists(file_path):
            df = pd.read_excel(file_path)
            df["ticker"] = ticker  # thêm cột ticker
            df_all.append(df)
        else:
            print(f"⚠️ Không tìm thấy: {file_path}")

    # Gộp lại thành 1 DataFrame
    df_global = pd.concat(df_all, ignore_index=True)
    print(f"✅ Đã load xong {len(df_global)} dòng dữ liệu từ {len(df_all)} cổ phiếu.")
    return df_global

In [32]:
tickers = ['ACB', 'BCM', 'BID','BVH','CTG','FPT','GAS','GVR','HDB','HPG',
                    'LPB','MBB','MSN','MWG','PLX','SAB','SHB','SSB','SSI','STB',
                    'TCB','TPB','VCB','VHM','VIB','VIC','VJC','VNM','VPB','VRE']
folder_path_train = "/kaggle/input/vn30-dataset/v6 - Copy/stock_trend/train_data" 
folder_path_test = "/kaggle/input/vn30-dataset/v6 - Copy/stock_trend/test_data" 
df_global_train = load_data_for_analysis(tickers, folder_path_train)
df_global_test = load_data_for_analysis(tickers, folder_path_test)


✅ Đã load xong 35865 dòng dữ liệu từ 30 cổ phiếu.
✅ Đã load xong 7380 dòng dữ liệu từ 30 cổ phiếu.


In [33]:
# Danh sách các mô hình
models = {
    "Random Forest": RandomForestClassifier(n_jobs=-1, random_state=42, class_weight='balanced'),
    "XGBoost": XGBClassifier(n_jobs=-1, random_state=42, use_label_encoder=False, eval_metric='mlogloss'),
    "LightGBM": LGBMClassifier(n_jobs=-1, random_state=42),
    "CatBoost": CatBoostClassifier(verbose=0, random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
}

In [34]:
X_train_no_sa = df_global_train[ta_fa_feature_selected]
X_test_no_sa = df_global_test[ta_fa_feature_selected]
y_train = df_global_train[target]
y_test = df_global_test[target]
scaler = StandardScaler()
X_train_no_sa_scaled = scaler.fit_transform(X_train_no_sa)
X_test_no_sa_scaled = scaler.transform(X_test_no_sa) 

In [35]:
X_train_with_sa = df_global_train[features]  # features = ta + fa + sa
X_test_with_sa = df_global_test[features]
y_train = df_global_train[target]
y_test = df_global_test[target]

scaler = StandardScaler()
X_train_with_sa_scaled = scaler.fit_transform(X_train_with_sa)   # Fit scaler trên train, scale train luôn
X_test_with_sa_scaled = scaler.transform(X_test_with_sa)  

In [36]:
# ==============================================Tuning with Optuna + TimeSeriesSplit validate (AUC macro) ===============================================

Divide 5 fold to cross validation though tuning parameters

In [37]:
def get_grouped_time_series_folds(df, ticker_col='ticker', n_splits=5):
    tickers = df[ticker_col].unique()
    fold_indices = [[] for _ in range(n_splits)]  # List chứa n_splits fold tổng
    for ticker in tickers:
        idx = df[df[ticker_col] == ticker].index.values
        tscv = TimeSeriesSplit(n_splits=n_splits)
        for fold_num, (train_idx, val_idx) in enumerate(tscv.split(idx)):
            # Gộp index thực tế vào fold tổng tương ứng
            fold_indices[fold_num].append((idx[train_idx], idx[val_idx]))
    # Gộp index của từng fold tổng lại thành mảng train/val duy nhất
    grouped_folds = []
    for fold in fold_indices:
        train_idx = np.concatenate([item[0] for item in fold])
        val_idx = np.concatenate([item[1] for item in fold])
        grouped_folds.append((train_idx, val_idx))
    return grouped_folds

In [38]:
folds = get_grouped_time_series_folds(df_global_train, ticker_col='ticker', n_splits=5)

Tuning random forest for stock trend with ta + fa data

In [39]:
X_no_sa = X_train_no_sa_scaled  # hoặc X_train_with_sa_scaled nếu dùng cả SA
y = y_train

In [40]:
print("Tuning XGBClassifier with Optuna (AUC macro) ta + fa features only")

def objective_xgb(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.7, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 0.9),
        "use_label_encoder": False,
        "eval_metric": "mlogloss",
        "random_state": 42,
        "tree_method": "hist",   # Sử dụng GPU
        "device": "cuda",
        "n_jobs": -1,
    }

    aucs = []
    for train_idx, val_idx in folds:
        X_train, X_val = X_no_sa[train_idx], X_no_sa[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
        model = XGBClassifier(**params)
        model.fit(X_train, y_train_fold)
        # One-hot encode y_val_fold
        y_val_bin = label_binarize(y_val_fold, classes=[0,1,2])
        y_pred_proba = model.predict_proba(X_val)
        # Nếu predict_proba trả về list (multioutput), cần stack lại
        if isinstance(y_pred_proba, list):
            y_pred_proba = np.stack([p[:,1] for p in y_pred_proba], axis=1)
        auc_macro = roc_auc_score(y_val_bin, y_pred_proba, average="macro", multi_class="ovr")
        aucs.append(auc_macro)
    mean_auc = np.mean(aucs)
    print(f"Trial {trial.number}: AUC macro={mean_auc:.5f}, params={trial.params}")
    # Optuna sẽ maximize AUC macro
    return -mean_auc

study = optuna.create_study(direction="minimize")
study.optimize(objective_xgb, n_trials=50)

print("Best AUC macro (XGBoost):", -study.best_value)
print("Best params (XGBoost):", study.best_trial.params)

[I 2025-06-20 06:33:39,574] A new study created in memory with name: no-name-72637669-f743-4b47-af3b-1aaf2e7bc739


Tuning XGBClassifier with Optuna (AUC macro) ta + fa features only


[I 2025-06-20 06:34:39,091] Trial 0 finished with value: -0.6290621703113837 and parameters: {'n_estimators': 361, 'max_depth': 14, 'learning_rate': 0.2427928908465474, 'subsample': 0.7046779369048635, 'colsample_bytree': 0.5416455900320072}. Best is trial 0 with value: -0.6290621703113837.


Trial 0: AUC macro=0.62906, params={'n_estimators': 361, 'max_depth': 14, 'learning_rate': 0.2427928908465474, 'subsample': 0.7046779369048635, 'colsample_bytree': 0.5416455900320072}


[I 2025-06-20 06:35:25,871] Trial 1 finished with value: -0.6390927197144833 and parameters: {'n_estimators': 419, 'max_depth': 9, 'learning_rate': 0.0674178265825021, 'subsample': 0.7929247630656305, 'colsample_bytree': 0.6806690252770973}. Best is trial 1 with value: -0.6390927197144833.


...

Trial 47: AUC macro=0.65499, params={'n_estimators': 746, 'max_depth': 12, 'learning_rate': 0.010426655483340486, 'subsample': 0.7172335199809252, 'colsample_bytree': 0.33652334829357333}


[I 2025-06-20 07:38:42,116] Trial 48 finished with value: -0.6547654506449738 and parameters: {'n_estimators': 745, 'max_depth': 12, 'learning_rate': 0.01081772884856319, 'subsample': 0.7008367131067763, 'colsample_bytree': 0.3402142721274026}. Best is trial 47 with value: -0.6549879905985198.


Trial 48: AUC macro=0.65477, params={'n_estimators': 745, 'max_depth': 12, 'learning_rate': 0.01081772884856319, 'subsample': 0.7008367131067763, 'colsample_bytree': 0.3402142721274026}


[I 2025-06-20 07:42:11,524] Trial 49 finished with value: -0.6548192664759828 and parameters: {'n_estimators': 839, 'max_depth': 12, 'learning_rate': 0.010738274497370262, 'subsample': 0.7170563915499023, 'colsample_bytree': 0.33233243900626563}. Best is trial 47 with value: -0.6549879905985198.


Trial 49: AUC macro=0.65482, params={'n_estimators': 839, 'max_depth': 12, 'learning_rate': 0.010738274497370262, 'subsample': 0.7170563915499023, 'colsample_bytree': 0.33233243900626563}
Best AUC macro (XGBoost): 0.6549879905985198
Best params (XGBoost): {'n_estimators': 746, 'max_depth': 12, 'learning_rate': 0.010426655483340486, 'subsample': 0.7172335199809252, 'colsample_bytree': 0.33652334829357333}


Tuning random forest for stock trend with ta + fa + sa data

In [41]:
X_with_sa = X_train_with_sa_scaled  
y = y_train

In [42]:
print("Tuning XGBClassifier with Optuna (AUC macro) ta + fa + sa features")

def objective_xgb(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 400, 1200),
        "max_depth": trial.suggest_int("max_depth", 4, 16),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "subsample": trial.suggest_float("subsample", 0.7, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 0.6),
        "use_label_encoder": False,
        "eval_metric": "mlogloss",
        "random_state": 42,
        "tree_method": "hist",   # Sử dụng GPU P100
        "device" : "cuda",
        "n_jobs": -1,
    }

    aucs = []
    for train_idx, val_idx in folds:
        X_train, X_val = X_with_sa[train_idx], X_with_sa[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
        model = XGBClassifier(**params)
        model.fit(X_train, y_train_fold)
        # One-hot encode y_val_fold
        y_val_bin = label_binarize(y_val_fold, classes=[0,1,2])
        y_pred_proba = model.predict_proba(X_val)
        # Nếu predict_proba trả về list (multioutput), cần stack lại
        if isinstance(y_pred_proba, list):
            y_pred_proba = np.stack([p[:,1] for p in y_pred_proba], axis=1)
        auc_macro = roc_auc_score(y_val_bin, y_pred_proba, average="macro", multi_class="ovr")
        aucs.append(auc_macro)
    mean_auc = np.mean(aucs)
    print(f"Trial {trial.number}: AUC macro={mean_auc:.5f}, params={trial.params}")
    # Optuna sẽ maximize AUC macro
    return -mean_auc

study = optuna.create_study(direction="minimize")
study.optimize(objective_xgb, n_trials=50)

print("Best AUC macro (XGBoost):", -study.best_value)
print("Best params (XGBoost):", study.best_trial.params)

[I 2025-06-20 07:42:11,547] A new study created in memory with name: no-name-4cecfff7-0b9f-42c1-9cdb-8cf91d71800e


Tuning XGBClassifier with Optuna (AUC macro) ta + fa + sa features


[I 2025-06-20 07:45:13,668] Trial 0 finished with value: -0.6604010332626717 and parameters: {'n_estimators': 1095, 'max_depth': 15, 'learning_rate': 0.1768230195554527, 'subsample': 0.7344455782479384, 'colsample_bytree': 0.49443341595412166}. Best is trial 0 with value: -0.6604010332626717.


Trial 0: AUC macro=0.66040, params={'n_estimators': 1095, 'max_depth': 15, 'learning_rate': 0.1768230195554527, 'subsample': 0.7344455782479384, 'colsample_bytree': 0.49443341595412166}


[I 2025-06-20 07:47:19,401] Trial 1 finished with value: -0.6744120894720327 and parameters: {'n_estimators': 1076, 'max_depth': 9, 'learning_rate': 0.03109978476488852, 'subsample': 0.9482155078784035, 'colsample_bytree': 0.5470478206611802}. Best is trial 1 with value: -0.6744120894720327.


...

Trial 47: AUC macro=0.67665, params={'n_estimators': 1083, 'max_depth': 9, 'learning_rate': 0.02288494971735982, 'subsample': 0.9458204239840304, 'colsample_bytree': 0.2747407720626561}


[I 2025-06-20 09:45:37,311] Trial 48 finished with value: -0.6625757700477569 and parameters: {'n_estimators': 624, 'max_depth': 10, 'learning_rate': 0.17945083443000479, 'subsample': 0.8869717741304081, 'colsample_bytree': 0.43926421898618895}. Best is trial 37 with value: -0.681833306455972.


Trial 48: AUC macro=0.66258, params={'n_estimators': 624, 'max_depth': 10, 'learning_rate': 0.17945083443000479, 'subsample': 0.8869717741304081, 'colsample_bytree': 0.43926421898618895}


[I 2025-06-20 09:47:23,829] Trial 49 finished with value: -0.6811449829787798 and parameters: {'n_estimators': 1167, 'max_depth': 8, 'learning_rate': 0.01017285432682622, 'subsample': 0.8330969679425234, 'colsample_bytree': 0.21446955200506454}. Best is trial 37 with value: -0.681833306455972.


Trial 49: AUC macro=0.68114, params={'n_estimators': 1167, 'max_depth': 8, 'learning_rate': 0.01017285432682622, 'subsample': 0.8330969679425234, 'colsample_bytree': 0.21446955200506454}
Best AUC macro (XGBoost): 0.681833306455972
Best params (XGBoost): {'n_estimators': 1089, 'max_depth': 8, 'learning_rate': 0.010501883895575981, 'subsample': 0.9005472962597326, 'colsample_bytree': 0.22675172202304014}
